In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype

# Mimic sklearn
## Normalization

In [2]:
def normalize(X): 
    if isinstance(X, pd.DataFrame):
        for c in X.columns:
            if is_numeric_dtype(X[c]):
                u = np.mean(X[c])
                s = np.std(X[c])
                X[c] = (X[c] - u) / s
        return
    
    for j in range(X.shape[1]):
        u = np.mean(X[:,j])
        s = np.std(X[:,j])
        X[:,j] = (X[:,j] - u) / s

## Linear Regression

In [3]:
def MSE(X,y,B,lmbda):
    """
    X, y, B are all numpy arrays
    X: (n, p) Matrix
    Y: (n, 1) vector
    B: (p, 1) coefficient vector
    In the final equation, we don’t care about scaling MSE by 1/n
    """
    dif = y - np.dot(X,B)
    return np.multiply(dif, dif)


def loss_gradient(X, y, B, lmbda):
    dif = y - np.dot(X,B)
    return -np.dot(np.transpose(X),dif)

**Adagrad Minimize**

In [4]:
def minimize(X, y, loss, loss_gradient,
             eta=0.00001, lmbda=0.0,
             max_iter=1000, addB0=True,
             precision=0.00000001):
    """
    eta: learning rate
    addB0: dictates whether or not to use augmented vectors and matrices. 
           We always set addB0=True except for regularizations, 
           which compute β0 outside of the minimization process (as just y) assuming standardized variables.
    """
    if X.ndim != 2:
        raise ValueError("X must be n x p for p features")
    n, p = X.shape
    if y.shape != (n, 1):
        raise ValueError(f"y must be n={n} x 1 not {y.shape}")

    B = np.random.random_sample(size=(p, 1)) * 2 - 1
    h = np.zeros(shape=(p,1))
    if addB0:
        n = X.shape[0]
        B0 = np.ones(shape=(n,1))
        X = np.hstack([B0, X]) # add column of 1s to X
        B = np.random.random_sample(size=(p+1, 1)) * 2 - 1  # make between [-1,1) 
        h = np.zeros(shape=(p+1,1))

    prev_B = B 
    step = 0
    eps = 1e-5 # prevent division by 0
    norm = np.linalg.norm(loss_gradient(X, y, B, lmbda))
    while step <= max_iter and norm > precision:
        cur_grad = loss_gradient(X, y, B, lmbda) 
        h += cur_grad * cur_grad
        B -= eta * cur_grad/(np.sqrt(h)+eps)
        step += 1
        norm = np.linalg.norm(loss_gradient(X, y, B, lmbda))
    return B

In [5]:
class LinearRegression_new: 
    def __init__(self, 
                 eta=0.00001, 
                 lmbda=0.0, 
                 max_iter=1000):
        self.eta = eta
        self.lmbda = lmbda
        self.max_iter = max_iter

    def fit(self, X, y):
        self.B = minimize(X, y,
                           MSE,
                           loss_gradient,
                           self.eta,
                           self.lmbda,
                           self.max_iter)
        
    def predict(self, X):
        n = X.shape[0]
        B0 = np.ones(shape=(n, 1))
        X = np.hstack([B0, X])
        return np.dot(X, self.B)

## L2 Regularization

In [6]:
def loss_ridge(X, y, B, lmbda):
    """
    Loss function for L2 regularization.
    We don’t care about scaling.
    """
    dif = y - np.dot(X,B)
    return np.multiply(dif,dif)+lmbda*np.multiply(B,B)


def loss_gradient_ridge(X, y, B, lmbda):
    dif = y - np.dot(X,B)
    return (-np.dot(np.transpose(X),dif)+lmbda*B)

In [7]:
class RidgeRegression_new: 
    def __init__(self,
                 eta=0.00001, lmbda=0.0,
                 max_iter=1000,addB0=False):
        self.eta = eta
        self.lmbda = lmbda
        self.max_iter = max_iter
        self.addB0 = addB0

    def predict(self, X):
        n = X.shape[0]
        B0 = np.ones(shape=(n, 1))
        X = np.hstack([B0, X])
        return np.dot(X, self.B)

    def fit(self, X, y):
        Bs = minimize(X, y,
                        loss_ridge,
                        loss_gradient_ridge,
                        self.eta,
                        self.lmbda,
                        self.max_iter,
                        self.addB0)
        B0 = np.mean(y)
        self.B = np.insert(Bs,0,B0).reshape(-1,1)

## Logistic Regression

In [8]:
def sigmoid(z):
    return 1/(1+np.exp(-z))


def log_likelihood(X, y, B,lmbda):
    term1 = sum(y*X) * B
    n = X.shape[0]
    s = 0
    for i in range(n):
        s += np.log(1+np.exp(np.dot(X[i,:],B)))
    return (-term1+s)


def log_likelihood_gradient(X, y, B, lmbda):
    dif = y - sigmoid(np.dot(X,B))
    return (-np.dot(np.transpose(X),dif))

In [9]:
class LogisticRegression_new: 
    def __init__(self,
                 eta=0.00001, lmbda=0.0,
                 max_iter=1000):
        self.eta = eta
        self.lmbda = lmbda
        self.max_iter = max_iter

    def fit(self, X, y):
        self.B = minimize(X, y,
                           log_likelihood,
                           log_likelihood_gradient,
                           self.eta,
                           self.lmbda,
                           self.max_iter)

    def predict_proba(self, X):
        """
        Compute the probability that the target is 1. 
        Do the usual linear regression and then pass through a sigmoid.
        """
        n = X.shape[0]
        B0 = np.ones(shape=(n, 1))
        X = np.hstack([B0, X])
        prob = sigmoid(np.dot(X,self.B))
        return prob

    def predict(self, X):
        """
        Call self.predict_proba() to get probabilities then, for each x in X,
        return a 1 if P(y==1,x) > 0.5 else 0.
        """     
        prob = self.predict_proba(X)
        def prob_p(x):
            if x > 0.5:
                return 1
            else:
                return 0
        pred = np.array(list(map(prob_p,prob)))
        return pred

# Test

## Linear Regression

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.datasets import load_boston

In [11]:
boston = load_boston()
X = boston.data
y = boston.target.reshape(-1, 1)
y.shape, X.shape

((506, 1), (506, 13))

In [12]:
normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=999)

In [13]:
lr = LinearRegression_new(max_iter=30000, eta=5)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("r^2", r2)

r^2 0.7022974023993411


## L2 Regularization

In [14]:
l2r = RidgeRegression_new(max_iter=30000, eta=5, lmbda=80)
l2r.fit(X_train,y_train)
y_pred = l2r.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("r^2", r2)

r^2 0.6573106705493537


## Logistic Regression

In [15]:
from test_class import *
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()
X = cancer.data
y = cancer.target.reshape(-1,1)
y.shape, X.shape

((569, 1), (569, 30))

In [16]:
normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=999)

In [17]:
logreg = LogisticRegression_new(max_iter=30000, eta=5,)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
print('accuracy:',accuracy_score(y_test, y_pred))

accuracy: 0.9385964912280702


In [18]:
logreg.predict_proba(X_test)

array([[1.00000000e+000],
       [5.71988721e-081],
       [2.76402440e-008],
       [2.27488510e-111],
       [1.00000000e+000],
       [9.95356113e-001],
       [1.00000000e+000],
       [6.34253990e-110],
       [1.00000000e+000],
       [1.27296053e-027],
       [1.00000000e+000],
       [1.00000000e+000],
       [4.63218756e-086],
       [1.00000000e+000],
       [1.00000000e+000],
       [1.00000000e+000],
       [1.00000000e+000],
       [1.00000000e+000],
       [1.00000000e+000],
       [5.90528248e-043],
       [1.44078087e-038],
       [1.00000000e+000],
       [2.40972862e-003],
       [1.86526911e-029],
       [9.99999746e-001],
       [1.08772275e-046],
       [1.34301912e-024],
       [4.48788480e-121],
       [1.00000000e+000],
       [1.67713689e-001],
       [1.50684610e-108],
       [9.61564896e-033],
       [1.00000000e+000],
       [1.00000000e+000],
       [1.00000000e+000],
       [4.29697181e-018],
       [9.99999995e-001],
       [1.00000000e+000],
       [4.22

In [19]:
logreg.predict(X_test)

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1])